- **语言**：Python 代码
- **目的**：从指定的 {folder} 中，找到包含关键词 {title_keywords} 的 PDF 文件，提取其中的 Figures 和 Tables，存入新建的 {output_folder} 文件夹。
- **输入**：
  - {folder}：包含 PDF 文件的文件夹路径。默认为当前路径。
  - {title_keywords}：用于匹配 PDF 文件名的关键词字符串。
- **提取规则：**
  - 基于 "Figure/Fig./Table + number" 等标记，确定图片的区域，截图导出 Figures 和 Tables。截图时，不要包含 Caption 和 Note 部分，只需截取图形和表格本身。注意：
    - 有些论文的 Figures 和 Tables 可能没有明确的 Caption 标题，此时可根据图形和表格的相对位置进行截图。
    - 截图时注意截取位置，不要截取到页眉、页脚等无关内容；也不要只截取部分图形或表格。
    - 截图时，四周留白 5-10 像素，以免图形或表格边缘被裁剪掉。
  - 对于比较长的表格 (超过页面 1/2 高度)，且包含多栏 ('Panel A.', 'Panel B', 或 'A. xx' 字样)，可以适当拆分为多张图片导出。
    - 每个子图的编号格式为 `{number}-{subnumber}`，如 Table 2-1、Figure 3-A 等。
  - 忽略：附录中的图形和表格不予提取。
  - 注意：只提取论文中的正式 Figures 和 Tables，不包括出版社的封面、封底等图片。
- **输出**：
  - output_folder = '{title_keywords}-out'。
    - 在当前路径下新建文件夹 {output_folder}，用于存放提取的 Figures 和 Tables。若已存在同名文件夹，则先删除再创建。
  - 将提取的 Figures 和 Tables 保存到指定的 {output_folder} 中，文件命名格式为
    - 主图： 
      - `{title_keywords}-Figure{number}.png` 
      - `{title_keywords}-Table{number}.png`
    - 子图：
      - `{title_keywords}-Figure{number}-{subnumber}.png`
      - `{title_keywords}-Table{number}-{subnumber}.png` 
  - 图片宽度：1000 像素，高度自适应。
- **依赖库** (建议)：
  - fitz (PyMuPDF)
  - PIL (Pillow)
  - os
  - re
  - shutil
- **代码运行方式**
  - 在 Jupyter Notebook 中运行。我会将你生成的代码复制到代码单元中。
  - 我会手动输入 {folder} 和 {title_keywords} 变量的值，然后运行代码单元。

In [4]:
# -*- coding: utf-8 -*-
"""
Jupyter Notebook 版本：
- 你只需要修改下面两个参数：folder、title_keywords
- 运行本 cell 后，会在当前工作目录下生成：{title_keywords}-out
- 输出图片宽度固定 1000 px，高度自适应
"""

import re
import shutil
from pathlib import Path

import fitz  # PyMuPDF


# -----------------------------
# 你只需要改这里
# -----------------------------
folder = r"."                 # PDF 文件所在文件夹路径，例如 r"D:\papers"
title_keywords = "Guo_2020_The_legacy_effect_of_unexploded_bombs_on_educational_attainment_in_Laos" # 用于匹配 PDF 文件名的关键词（不区分大小写）


# -----------------------------
# 内部实现：caption 检测与裁剪
# -----------------------------
FIG_PATTERN = re.compile(r"^\s*(figure|fig\.?)\s*([0-9]+)\b", re.IGNORECASE)
TAB_PATTERN = re.compile(r"^\s*(table)\s*([0-9]+)\b", re.IGNORECASE)


def iter_caption_lines(page: fitz.Page):
    """
    逐行产出 (line_text, line_rect)。
    使用 get_text("dict") 获取行级坐标，便于按 caption 位置裁剪截图区域。
    """
    d = page.get_text("dict")
    for block in d.get("blocks", []):
        # type=0 表示文本块
        if block.get("type", 1) != 0:
            continue
        for line in block.get("lines", []):
            spans = line.get("spans", [])
            if not spans:
                continue

            # 拼接该行文本
            line_text = "".join(s.get("text", "") for s in spans).strip()

            # 合并该行 bbox
            x0 = min(s["bbox"][0] for s in spans)
            y0 = min(s["bbox"][1] for s in spans)
            x1 = max(s["bbox"][2] for s in spans)
            y1 = max(s["bbox"][3] for s in spans)
            yield line_text, fitz.Rect(x0, y0, x1, y1)


def find_captions_on_page(page: fitz.Page):
    """
    找到本页中以 Figure/Fig./Table 开头的 caption 行，并返回列表：
    [{"kind": "Figure"/"Table", "num": int, "rect": Rect, "text": str}, ...]
    """
    lines = list(iter_caption_lines(page))
    caps = []

    i = 0
    while i < len(lines):
        text, rect = lines[i]

        m_fig = FIG_PATTERN.match(text)
        m_tab = TAB_PATTERN.match(text)

        if not (m_fig or m_tab):
            i += 1
            continue

        if m_fig:
            kind = "Figure"
            num = int(m_fig.group(2))
        else:
            kind = "Table"
            num = int(m_tab.group(2))

        # caption 可能跨行：做一个简单合并（相邻行 y 距离近、缩进相近、且不出现新的 caption）
        merged_text = text
        merged_rect = fitz.Rect(rect)

        j = i + 1
        while j < len(lines):
            nxt_text, nxt_rect = lines[j]

            close_in_y = (nxt_rect.y0 - merged_rect.y1) <= 12
            not_new_caption = not (FIG_PATTERN.match(nxt_text) or TAB_PATTERN.match(nxt_text))
            similar_indent = abs(nxt_rect.x0 - merged_rect.x0) <= 30

            if close_in_y and not_new_caption and similar_indent:
                merged_text = (merged_text + " " + nxt_text).strip()
                merged_rect |= nxt_rect
                j += 1
            else:
                break

        caps.append({"kind": kind, "num": num, "rect": merged_rect, "text": merged_text})
        i = j

    return caps


def clip_by_caption(page: fitz.Page, cap: dict):
    """
    基于 caption 的 bbox 生成截图区域 clip（启发式规则）：
    - Figure：通常“图在上、caption 在下”
    - Table：通常“caption 在上、表在下”
    """
    r = cap["rect"]
    pr = page.rect
    H = pr.height

    margin_x = 20
    margin_y = 10

    if cap["kind"] == "Figure":
        # 往上抓更多
        y0 = max(pr.y0, r.y0 - 0.55 * H)
        y1 = min(pr.y1, r.y1 + 0.10 * H)
    else:
        # 往下抓更多
        y0 = max(pr.y0, r.y0 - 0.10 * H)
        y1 = min(pr.y1, r.y1 + 0.55 * H)

    # 横向优先抓全栏（论文图表通常较宽）
    x0 = max(pr.x0, r.x0 - margin_x)
    x1 = min(pr.x1, r.x1 + (pr.width * 0.90))

    # 若横向太窄，直接扩到近似全宽
    if (x1 - x0) < 0.60 * pr.width:
        x0 = pr.x0 + 10
        x1 = pr.x1 - 10

    clip = fitz.Rect(x0, y0 - margin_y, x1, y1 + margin_y)
    clip = clip & pr
    return clip


def render_clip_to_png(page: fitz.Page, clip: fitz.Rect, out_png: Path, target_width_px: int = 1000):
    """
    渲染 clip 到 PNG：
    - 输出宽度固定 target_width_px
    - 高度自适应
    """
    rect_w = max(1.0, clip.width)

    # 缩放倍数：使输出宽度接近 1000 px
    zoom = target_width_px / rect_w

    # 限制 zoom，避免极端情况下过慢或过糊
    zoom = max(1.0, min(zoom, 6.0))

    mat = fitz.Matrix(zoom, zoom)
    pix = page.get_pixmap(matrix=mat, clip=clip, alpha=False)
    pix.save(str(out_png))


def extract_figures_tables(folder: str, title_keywords: str, recursive: bool = False):
    """
    主函数：
    - 在 folder 下查找文件名包含 title_keywords 的 PDF
    - 创建输出目录 {title_keywords}-out（若存在则删除重建）
    - 抽取 Figure/Table 截图并保存 PNG
    """
    base = Path(folder).expanduser().resolve()
    if not base.exists():
        raise FileNotFoundError(f"folder 不存在：{base}")

    # 查找 PDF（可选递归）
    kw = title_keywords.lower()
    if recursive:
        pdf_files = sorted([p for p in base.rglob("*.pdf") if kw in p.name.lower()])
    else:
        pdf_files = sorted([p for p in base.glob("*.pdf") if kw in p.name.lower()])

    if not pdf_files:
        print(f"未找到匹配的 PDF：folder={base}, title_keywords={title_keywords}")
        return

    # 输出目录：在当前工作目录下创建
    output_folder = Path.cwd() / f"{title_keywords}-out"
    if output_folder.exists():
        shutil.rmtree(output_folder)
    output_folder.mkdir(parents=True, exist_ok=True)

    # 去重集合，避免同页重复匹配同一个 caption
    fig_seen = set()
    tab_seen = set()

    print(f"匹配到 {len(pdf_files)} 个 PDF：")
    for p in pdf_files:
        print(f"  - {p.name}")

    for pdf_path in pdf_files:
        doc = fitz.open(str(pdf_path))
        try:
            for page_index in range(doc.page_count):
                page = doc.load_page(page_index)
                caps = find_captions_on_page(page)

                for cap in caps:
                    clip = clip_by_caption(page, cap)

                    # 去重 key：同一 PDF、同一编号、同一页、caption 起始 y 相近
                    key = (pdf_path.name, cap["num"], page_index, round(cap["rect"].y0, 1))

                    if cap["kind"] == "Figure":
                        if key in fig_seen:
                            continue
                        fig_seen.add(key)
                        out_name = f"{title_keywords}-Figure{cap['num']}.png"
                    else:
                        if key in tab_seen:
                            continue
                        tab_seen.add(key)
                        out_name = f"{title_keywords}-Table{cap['num']}.png"

                    out_png = output_folder / out_name

                    # 若同名已存在（例如多个 PDF 都有 Figure 1），自动追加后缀避免覆盖
                    if out_png.exists():
                        stem = out_png.stem
                        suffix = out_png.suffix
                        k = 2
                        while True:
                            cand = output_folder / f"{stem}-{k}{suffix}"
                            if not cand.exists():
                                out_png = cand
                                break
                            k += 1

                    render_clip_to_png(page, clip, out_png, target_width_px=1000)

        finally:
            doc.close()

    print(f"完成。输出目录：{output_folder}")


# -----------------------------
# 运行
# -----------------------------
extract_figures_tables(folder=folder, title_keywords=title_keywords, recursive=False)


匹配到 1 个 PDF：
  - Guo_2020_The_legacy_effect_of_unexploded_bombs_on_educational_attainment_in_Laos.pdf
完成。输出目录：d:\JG\李晓燕共享\已完成\2024已完成\2024-1-寒假\refs\C-refs-Guo\Guo_2020_The_legacy_effect_of_unexploded_bombs_on_educational_attainment_in_Laos-out
